# EDA and Prediction of Indian Housing price

# Context
This data set is created only for the learning purpose of the customer segmentation concepts , also known as house price prediction . I will demonstrate this by using supervised ML technique in the simplest form.

Source: Kaggle
Url: https://www.kaggle.com/himanshuntt/indian-housing-price

### Importing PyForest to avoid the hassel of importing all the libraries. Dataset is being imported

In [1]:
#!pip install PyForest
#from pyforest import *
#ELSE import required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('../input/indian-housing-price/houseprice.csv')

### Data spread and diversites inspection

In [3]:
df.head()

,Living Area,Bathrooms,Bedrooms,Lot Size,Age,Fireplace,Price
0,1.982,1.0,3,2.00,133,0,14.2212
1,1.676,1.5,3,0.38,14,1,13.4865
2,1.694,2.0,3,0.96,15,1,11.8007
3,1.800,1.0,2,0.48,49,1,13.8297
4,2.088,1.0,3,1.84,29,1,12.9470


In [4]:
df.tail()

,Living Area,Bathrooms,Bedrooms,Lot Size,Age,Fireplace,Price
1042,1.802,2.0,4,0.97,56,1,10.7695
1043,3.239,3.5,4,2.50,1,1,23.6737
1044,1.440,2.0,2,0.61,66,1,15.4829
1045,2.030,2.5,3,1.00,3,1,17.9492
1046,2.097,2.5,3,1.93,10,1,18.9108


Observation: Most of the features are are straight forward to understand and the spread of the data in the top and bottom of dataset seems to be distributed evenly

### Finding the shape and data types in dataset

df.shape

Observation: data set consists of 1047 rows/observation and 7 attributes/features

In [ ]:
df.info()

Features are all numerical including floating and integer type

### Statistical analysis of dataset and finding null values

In [ ]:
df.describe()

Observation: Outliers seems to be in lot size and Age features. Rest of the features has values which are not much deviating from mean.

In [ ]:
df.isnull().sum()/df.shape[1]

Observation: There sre no missing values to handle.

# Visualization

In [ ]:
plt.subplots(figsize=(6,6))
plt.plot()
sns.boxplot(df.Age, orient='h')

In [ ]:
sns.distplot(df.Age)

Inference: Age feature consists of outilers and it mostly right skewed which needs to be handled

In [ ]:
data=df.drop(['Age','Price'], axis=1)

In [ ]:
sns.boxplot(data=data, orient='v')

In [ ]:
sns.distplot(df['Lot Size'])

In [ ]:
sns.distplot(df['Living Area'])

Inference: Lot Size and living area has outliers and mostly right skewed which needs to be handled

### Scaling features to handle outliers instead of using IQR or z score methods as there are only 1000+ obersvations to build best model

### Age

In [ ]:
from sklearn.preprocessing import FunctionTransformer
ft=FunctionTransformer(np.log)
df['Age_log']=ft.fit_transform(df[['Age']])

### Lot Size

In [ ]:
ff=FunctionTransformer(np.log)
df['LotSize_log']=ft.fit_transform(df[['Lot Size']])

### Living Area

In [ ]:
ff=FunctionTransformer(np.log)
df['Living_log']=ft.fit_transform(df[['Living Area']])

In [ ]:
df.columns

In [ ]:
plt.subplots(figsize=(6,6))
sns.boxplot(df.Age_log, orient='h')

In [ ]:
plt.subplots(figsize=(6,6))
sns.boxplot(df.LotSize_log, orient='h')

In [ ]:
plt.subplots(figsize=(6,6))
sns.boxplot(df.Living_log, orient='h')

Inference: Age and living area feature's outliers are handled but Lot size consists of outliers

### Replacing 'inf' values with Nan and dropping values

In [ ]:
df.replace(-np.inf, np.nan, inplace=True)
df.dropna(axis=0, inplace=True)

In [ ]:
sns.distplot(df.Age_log)

Inference: Age_log feature doesn't contain outliers but graph is not normal. Model can be built and then its wise to mtake a decision on this feature

In [ ]:
sns.distplot(df.LotSize_log)

Inference: Lot Size still consists of outliers but almost equal to normal graph. Model can be built and then its wise to mtake a decision on this feature

In [ ]:
sns.distplot(df.Living_log)

Inference: Living area does not consists of outliers and almost equal to normal graph. Model can be built and then its wise to mtake a decision on this feature

In [ ]:
sns.jointplot(df.Age_log, df.Price)

Inference: Varied price range can be observed for less aged houses but price ranges are little low for aged houses.

In [ ]:
data=df.drop(['Age','Lot Size','Living Area'], axis=1)
g=sns.pairplot(data=data)

Inference: Price Appears o have linear relationship with living log and inversly related with Age

In [ ]:
sns.countplot(df.Bedrooms)

Inference: There are maximum number of 3 bedroom houses

In [ ]:
sns.violinplot(df.Fireplace, df.Price)

Inference: House price is lesser for houses without fireplace when compared to those which have

In [ ]:
sns.barplot(df.Bedrooms, df.Price, hue=df.Fireplace, palette='autumn')

Inference:Single bedroom with/without fireplace costs the same but as the number of bedrooms increases, with fireplace costs more than without.

In [ ]:
sns.swarmplot(df.Bathrooms, df.Price)

Inference: Houses with lesser number of bathrooms have less price. But the range of price spread is greater with 1.5 to 2.5 bathrooms.

# Cleaned Dataset ready for prediction

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

## Dataset with transfomed features which helps to predict good model

In [ ]:
df.drop(['Age','Lot Size','Living Area'], axis=1, inplace= True)

In [ ]:
df.head()

# Initial LR model

In [ ]:
x=df.drop(['Price'], axis=1)
y=df['Price']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=.3, random_state=1 )

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train,y_train)
print("coefficients are ",lr.coef_,"intercept is ", lr.intercept_)
print()
print('r2 score for training data: ',r2_score(y_train, lr.predict(X_train)))
print('r2 score for testing data: ',r2_score(y_test, lr.predict(X_test)))
print('rmse score: ',np.sqrt(mean_squared_error(y_train, lr.predict(X_train))))

##### Inference: Cost function seems high and tunning of the model is necessary to get a better r2 score. Model seems overfitting

# OLS Model pearson correlation values for features

In [ ]:
import statsmodels.api as sm
xc=sm.add_constant(x)
model=sm.OLS(y,xc).fit()
model.summary()

###### Observation: JB value is higher and Bedrooms p-value is more than level of significance. Target is lightly skewed with peakedness. Features needs to be tuned to build better model

# Checking for assumptions:

### 1. No auto correlation 

In [ ]:
import statsmodels.tsa.api as smt
pattern=smt.graphics.plot_acf(model.resid, lags=40)
pattern.show()

Inference: pattern is neither cyclic nor alternative. DB constant is about 1.62 which exhibits a min correlation between residuals

### 2. Normality of residuals

In [ ]:
from scipy.stats import norm
sns.distplot(model.resid, fit=norm)

In [ ]:
import scipy.stats as st
st.jarque_bera(model.resid)

Inference: p-value should be greater than .05 but the value shows 0.0, therefore feature scaling is necessary

In [ ]:
x=df.drop(['Price'], axis=1)
x=x.transform(lambda x: x**2)
y=df['Price'].transform(lambda x: x**(1/3))

In [ ]:
import statsmodels.api as sm
xc=sm.add_constant(x)
model=sm.OLS(y,xc).fit()
model.summary()

In [ ]:
st.jarque_bera(model.resid)

Inference: Residuals are not following normality. Hence the assumption is not satisfied

### 3. Linearity in residuals

In [ ]:
sns.regplot(x=y, y=model.predict(), lowess=True, line_kws={'color':'red'})
plt.xlabel('y actual')
plt.ylabel('y prediction')

In [ ]:
st.probplot(model.resid, plot=plt)
plt.show()

In [ ]:
sm.stats.linear_rainbow(res=model)

Inference: As p-value> .05, the residuals appears to be linear

### 4. Homoscedacity of residuals

In [ ]:
sns.regplot(x=model.predict(), y=model.resid, lowess=True, line_kws={'color':'red'})

In [ ]:
import statsmodels.stats.api as stt
stt.het_goldfeldquandt(model.resid, model.model.exog)

Inference: As p-value is greater than .05, residuals are homoscedasctic in nature

### 5. Multicolinearity between features

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif=[variance_inflation_factor(x.values,i) for i in range(x.shape[1])]
v=pd.DataFrame({'vif':vif[:]}, index=x.columns)

In [ ]:
v.T

In [ ]:
sns.heatmap(df.corr(), annot=True)

Inference: VIF seems low for all variables and only correlation exists between bathrooms and living area which will be handled

# Models

## Recursive method of elimination

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
no_of_features=np.arange(1,7)
highscore=0
score_list=[]
x=df.drop(['Price'], axis=1)
x=x.transform(lambda x: x**2)
y=df['Price'].transform(lambda x: x**(1/3))
for i in range(len(no_of_features)):
    X_train,X_test,y_train,y_test=train_test_split(x,y, test_size=.2, random_state=1)
    model=LinearRegression()
    rfe=RFE(model, no_of_features[i])
    X_rfe_train=rfe.fit_transform(X_train,y_train)
    X_rfe_test=rfe.transform(X_test)
    model.fit(X_rfe_train,y_train)
    score=model.score(X_rfe_test,y_test)
    print(score, end=' ')
    score_list.append(score)
    if score>highscore:
        highscore=score
        nof=no_of_features[i]
print()        
print("Optimum number of feature to be selected is ", nof, " and its r2 is ",highscore)

##### Inference: The r2 score is with 59% accuracy. recursiveve method of elimination cannot be considered

# VIF

In [ ]:
thres=5.0
op=pd.DataFrame()
x=df.drop(['Price'], axis=1)
x=x.transform(lambda x: x**2)
y=df['Price'].transform(lambda x: x**(1/3))
k=len(x.columns)
vif=[variance_inflation_factor(x.values, i) for i in range(x.shape[1])]
for j in range(1,k+1):
    print('iteration num ',j)
    print(vif)
    a=np.argmax(vif)
    print("the variable number is", a)
    if vif[a]<=thres:
        break
    elif j==1:
        op=x.drop(x.columns[a], axis=1)
        vif=[variance_inflation_factor(op.values,i) for i in range(op.shape[1])]
    elif j>1:
        op=op.drop(op.columns[a], axis=1)
        vif=[variance_inflation_factor(op.values,i) for i in range(op.shape[1])]       
op

In [ ]:
x=df.drop(['Price','Bedrooms','Bathrooms'], axis=1)
x=x.transform(lambda x: x**2)
y=df['Price'].transform(lambda x: x**(1/3))
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=.2, random_state=1 )
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train,y_train)
print("coefficients are ",lr.coef_,"intercept is ", lr.intercept_)
print()
print('r2 score of train is: ',r2_score(y_train, lr.predict(X_train)))
print('r2 score of test is: ',r2_score(y_test, lr.predict(X_test)))
print('rmse score: ',np.sqrt(mean_squared_error(y_train, lr.predict(X_train))))

##### Inference: The r2 score has been reduced. VIF method of elimination can be considered .

## Linear Regression model with Backward elimination methodoloy

In [ ]:
x=df.drop(['Price'], axis=1)
x=x.transform(lambda x: x**2)
y=df['Price'].transform(lambda x: x**(1/3))


import statsmodels.api as sm
cols=list(x.columns)
pmax=1
while(len(cols)>0):
    p=[]
    x_1=x[cols]
    x_1=sm.add_constant(x_1)
    model=sm.OLS(y,x_1).fit()
    p=pd.Series(model.pvalues.values[1:], index=cols)
    pmax=max(p)
    feature_of_pmax=p.idxmax()
    if (pmax>.05):
        cols.remove(feature_of_pmax)
    else:
        break;
selected_feature_BE=cols
print(selected_feature_BE)

In [ ]:
####As per backward elimination, Bedrooms and LotSize_log features are ommitted

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
x=df.drop(['Price','LotSize_log','Bedrooms'], axis=1)
x=x.transform(lambda x: x**2)
y=df['Price'].transform(lambda x: x**(1/3))
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=.2, random_state=1 )
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train,y_train)
print("coefficients are ",lr.coef_,"intercept is ", lr.intercept_)
print()
print('r2 score for training data: ',r2_score(y_train, lr.predict(X_train)))
print('r2 score for testing data: ',r2_score(y_test, lr.predict(X_test)))
print('rmse score: ',np.sqrt(mean_squared_error(y_train, lr.predict(X_train))))

##### Inference: The RMSE rate has been reduced but the model has 60% accuracy

# Ridge and Lasso with Grid Search Cross validation

### Ridge

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge,RidgeCV, Lasso,LassoCV
x=df.drop(['Price','Bedrooms','LotSize_log'], axis=1)
x=x.transform(lambda x: x**2)
y=df['Price'].transform(lambda x: x**(1/3))
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=.2, random_state=1 )
alpha=[1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20,50,100]
ridge=Ridge()
parameters={'alpha':alpha}
ridge_regressor=GridSearchCV(ridge,parameters,scoring='neg_mean_squared_error',cv=5)
ridge_regressor.fit(X_train,y_train)

In [ ]:
r_rmse=np.sqrt(mean_squared_error(ridge_regressor.predict(X_test), y_test))
print(ridge_regressor.best_params_, "and RMSE:", r_rmse)
print("r2 score train: ",r2_score(y_train, ridge_regressor.predict(X_train)), 
                                  "r2 score test: ",r2_score(y_test, ridge_regressor.predict(X_test)))

### Lasso 

In [ ]:
alpha=[1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20,50,100]
lasso=Lasso()
x=df.drop(['Price','LotSize_log','Bedrooms'], axis=1)
x=x.transform(lambda x: x**2)
y=df['Price'].transform(lambda x: x**(1/3))
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=.2, random_state=1 )
parameters={'alpha':alpha}
lasso_regressor=GridSearchCV(lasso,parameters,scoring='neg_mean_squared_error',cv=5)
lasso_regressor.fit(X_train,y_train)

In [ ]:
l_rmse=np.sqrt(mean_squared_error(lasso_regressor.predict(X_test), y_test))
print(lasso_regressor.best_params_, "and RMSE:", l_rmse)
print("r2 score train: ",r2_score(y_train, lasso_regressor.predict(X_train)), 
                                  "r2 score test: ",r2_score(y_test, lasso_regressor.predict(X_test)))

# Final predictions with LR, Ridge and Lasso models

In [ ]:
LR=lr.predict(X_test)
RR=ridge_regressor.predict(X_test)
LL=lasso_regressor.predict(X_test)
actual=y_test.values
FinalDF=pd.DataFrame(actual, columns=['Actual'])
FinalDF['LR prediction']=LR
FinalDF['Ridge prediction']=RR
FinalDF['Lasso Prediction']=LL

In [ ]:
FinalDF

In [ ]:
fig, ax = plt.subplots(2,1 , figsize=(15,10))
price_head=FinalDF.head(30)
price_head.plot(kind='bar', ax=ax[0])
price_tail=FinalDF.tail(30)
price_tail.plot(kind='bar', ax=ax[1])

#### Inference: 60% of accuracy can be obtained from all the three models with 0.21 RMSE value 

# Final scores of different models applied

In [ ]:
r2_train=[r2_score(y_train, lr.predict(X_train)), r2_score(y_train, lasso_regressor.predict(X_train)),r2_score(y_train, lasso_regressor.predict(X_train))]
r2_test=[r2_score(y_test, lr.predict(X_test)),r2_score(y_test, lasso_regressor.predict(X_test)),r2_score(y_test, ridge_regressor.predict(X_test))]
rmse=[np.sqrt(mean_squared_error(lr.predict(X_test), y_test)),np.sqrt(mean_squared_error(lasso_regressor.predict(X_test), y_test)),np.sqrt(mean_squared_error(ridge_regressor.predict(X_test), y_test))]

In [ ]:
FinalScores=pd.DataFrame(r2_train, columns=['r2 train'], index=['LR','Lasso','Ridge'])
FinalScores['r2 test']=r2_test
FinalScores['RMSE']=rmse

In [ ]:
FinalScores